<a href="https://colab.research.google.com/github/Volodikova/volodikova/blob/main/%D0%BC%D0%BE%D0%B4%D0%B5%D0%BB%D1%8C_%D0%BF%D0%BE_%D1%81%D1%82%D1%80%D0%B0%D1%85%D0%BE%D0%B2%D0%BA%D0%B5_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Предсказываем стоимость мед страховки



## Загужаем необходимые библиотеки

In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

## Загружаем данные

Собраны данные:
- age: возраст
- sex: пол
- bmi: индекс массы тела
- children: количество детей, охваченных медицинским страхованием / количество иждивенцев
- smoker: курение
- region: регион (northeast, southeast, southwest, northwest).
- charges: индивидуальные медицинские расходы (его и хотим предсказать)

In [ ]:
# Загрузите данные из файла insurance.csv в переменную df


In [2]:
df=pd.read_csv('insurance.csv')

In [3]:
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


## Смотрим статистику, что нет пропусков и отсуствующих значений

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [5]:
df.describe()

,age,bmi,children,charges
count,1338.000000,1338.000000,1338.000000,1338.000000
mean,39.207025,30.663397,1.094918,13270.422265
std,14.049960,6.098187,1.205493,12110.011237
min,18.000000,15.960000,0.000000,1121.873900
25%,27.000000,26.296250,0.000000,4740.287150
50%,39.000000,30.400000,1.000000,9382.033000
75%,51.000000,34.693750,2.000000,16639.912515
max,64.000000,53.130000,5.000000,63770.428010


## Преобразуем строковые данные

In [6]:
# Заменяем пол и курение на числа
df['sex']=df['sex'].map({'male':1, 'female':0})
df['smoker']=df['smoker'].map({'yes':1,'no':0})

In [7]:
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,0,27.900,0,1,southwest,16884.92400
1,18,1,33.770,1,0,southeast,1725.55230
2,28,1,33.000,3,0,southeast,4449.46200
3,33,1,22.705,0,0,northwest,21984.47061
4,32,1,28.880,0,0,northwest,3866.85520


In [8]:
# Заменяем регион на набор отдельных колонок (is_southwest, is_southeast и тп)
df = pd.get_dummies(df, columns=['region'])
df.head()

,age,sex,bmi,children,smoker,charges,region_northeast,region_northwest,region_southeast,region_southwest
0,19,0,27.900,0,1,16884.92400,0,0,0,1
1,18,1,33.770,1,0,1725.55230,0,0,1,0
2,28,1,33.000,3,0,4449.46200,0,0,1,0
3,33,1,22.705,0,0,21984.47061,0,1,0,0
4,32,1,28.880,0,0,3866.85520,0,1,0,0


In [10]:
print(df.describe)

<bound method NDFrame.describe of       age  sex     bmi  ...  region_northwest  region_southeast  region_southwest
0      19    0  27.900  ...                 0                 0                 1
1      18    1  33.770  ...                 0                 1                 0
2      28    1  33.000  ...                 0                 1                 0
3      33    1  22.705  ...                 1                 0                 0
4      32    1  28.880  ...                 1                 0                 0
...   ...  ...     ...  ...               ...               ...               ...
1333   50    1  30.970  ...                 1                 0                 0
1334   18    0  31.920  ...                 0                 0                 0
1335   18    0  36.850  ...                 0                 1                 0
1336   21    0  25.800  ...                 0                 0                 1
1337   61    0  29.070  ...                 1                 0 

## Формируем признаки и целевую переменную

In [ ]:
## Сформируйте признаки и целевую переменную

In [11]:
target_value=df['charges']

In [15]:
x=df[['age','sex','bmi','children','smoker','region_northwest','region_northeast','region_southeast','region_southwest']]
y=target_value

In [17]:
x

,age,sex,bmi,children,smoker,region_northwest,region_northeast,region_southeast,region_southwest
0,19,0,27.900,0,1,0,0,0,1
1,18,1,33.770,1,0,0,0,1,0
2,28,1,33.000,3,0,0,0,1,0
3,33,1,22.705,0,0,1,0,0,0
4,32,1,28.880,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...
1333,50,1,30.970,3,0,1,0,0,0
1334,18,0,31.920,0,0,0,1,0,0
1335,18,0,36.850,0,0,0,0,1,0
1336,21,0,25.800,0,0,0,0,0,1


In [18]:
y

0       16884.92400
1        1725.55230
2        4449.46200
3       21984.47061
4        3866.85520
           ...     
1333    10600.54830
1334     2205.98080
1335     1629.83350
1336     2007.94500
1337    29141.36030
Name: charges, Length: 1338, dtype: float64

## Разделяем данные на выборку для обучения/проверки

In [39]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.1)

In [40]:
x_train

,age,sex,bmi,children,smoker,region_northwest,region_northeast,region_southeast,region_southwest
341,62,1,30.02,0,0,1,0,0,0
839,59,0,31.35,0,0,1,0,0,0
1211,39,1,34.10,2,0,0,0,1,0
983,27,0,30.59,1,0,0,1,0,0
47,28,0,34.77,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...
1274,26,1,27.06,0,1,0,0,1,0
594,41,1,40.26,0,0,0,0,1,0
646,39,1,26.22,1,0,1,0,0,0
927,57,0,20.10,1,0,0,0,0,1


In [41]:
x_train.info

<bound method DataFrame.info of       age  sex    bmi  ...  region_northeast  region_southeast  region_southwest
341    62    1  30.02  ...                 0                 0                 0
839    59    0  31.35  ...                 0                 0                 0
1211   39    1  34.10  ...                 0                 1                 0
983    27    0  30.59  ...                 1                 0                 0
47     28    0  34.77  ...                 0                 0                 0
...   ...  ...    ...  ...               ...               ...               ...
1274   26    1  27.06  ...                 0                 1                 0
594    41    1  40.26  ...                 0                 1                 0
646    39    1  26.22  ...                 0                 0                 0
927    57    0  20.10  ...                 0                 0                 1
416    52    1  34.10  ...                 0                 1               

In [29]:
y_test

924      6250.43500
475     28868.66390
745      9910.35985
408      6652.52880
1250    18648.42170
           ...     
56      13607.36875
79       6571.02435
1290     7133.90250
783     24520.26400
802      2103.08000
Name: charges, Length: 402, dtype: float64

## Создаем и обучаем модель линейной регресии

In [ ]:
# Создайте и обучите модель

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [42]:
lr=LinearRegression()
lr.fit(x_train,y_train)
LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)
lr.coef_

array([  253.5709532 ,  -157.81543853,   339.53237031,   519.18887596,
       24066.87755307,   233.44851168,   368.31064701,  -423.12194506,
        -178.63721362])

## Получаем предсказание и оцениваем качество

In [ ]:
# Получите предсказание

In [43]:
lr.predict(x_test)

array([26753.29718181, 29042.42855599,  3322.36607417, 11984.10470167,
        7549.35594471, 12569.27473206,  8752.55176917,  5679.75997109,
        6494.29810404, 15042.0762641 ,  6887.58048766, 13034.26090414,
       11563.74449483,  6809.52058255, 14328.37703359,  2310.82242289,
       -1080.32837641,  2995.55443583, 17412.84941217, 11893.13736845,
       31850.36304211,  1857.51541973, 12584.24907864,  7201.64646117,
        6278.83544322, 13151.69673997,  2380.81843235, 34143.79243519,
        2998.07683566, 11707.23325853,  6644.75370222, 27464.86491716,
       10866.35749498,  9393.07574884,  4422.15530337, 40139.63100223,
        5692.75023088, 10646.32064401,  8820.41916567,  6548.46293318,
        2220.48059399, 34189.33154725, 10954.17692405,   269.63648228,
        1656.44145358,  4924.23810365, 10421.79327983, 10612.77875301,
       14408.84090302, 15125.29778589, 12627.42383793, 11631.37077011,
       13317.43217397, 10275.50704347,  8313.50408474,  9894.27997592,
      

In [34]:
#оценка на глаз

In [44]:
pd.DataFrame([y_test.values,lr.predict(x_test)]).T

,0,1
0,16115.30450,26753.297182
1,18765.87545,29042.428556
2,12609.88702,3322.366074
3,19144.57652,11984.104702
4,8252.28430,7549.355945
...,...,...
129,5028.14660,7067.266845
130,5979.73100,9437.350257
131,24106.91255,34059.649952
132,13143.86485,14739.332740


In [ ]:
# Оцените качество, при помощи метода mean_squared_error для тестовой выборки

In [45]:
mean_squared_error(y_test,lr.predict(x_test))

37941939.64068102

In [46]:
mean_squared_error(y_train,lr.predict(x_train))

36380111.95468785

## Делаем предсказание для одного человека

In [47]:
# Заполняем данные по конкретному человеку
data = [{
    "age": 20,
    "sex": 1,
    "bmi": 30,
    "children": 2,
    "smoker": 1,
    "region_northeast": 0,
    "region_northwest": 0,
    "region_southeast": 1,
    "region_southwest": 0
}]

In [48]:
df_person = pd.DataFrame(data)
df_person.head()

,age,sex,bmi,children,smoker,region_northeast,region_northwest,region_southeast,region_southwest
0,20,1,30,2,1,0,0,1,0


In [49]:
lr.predict(df_person)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:488: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


array([27327.47714235])